In [1]:
# Se cargan todas las librerías
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
import xgboost as xgb
import lightgbm as lgbm
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV, cross_validate, cross_val_score
from sklearn.decomposition import PCA

## DATASET 1: CONSUMO F&H

In [2]:
# Se cargan los dos datasets (producción y consumo)
df_consumo = pd.read_csv("consumoF&H_Limpio.csv", encoding = 'utf-8-sig')

In [3]:
df_consumo

,Fecha,Año,Mes,CCAA,Producto,Volumen (miles de kg),Valor (miles de €),Precio medio kg,Penetración (%),Consumo per capita,Gasto per capita
0,03-2018,2018,3,Total Nacional,TOTAL PATATAS,111978.64,92382.14,0.82,80.93,2.45,2.02
1,03-2018,2018,3,Total Nacional,PATATAS FRESCAS,79299.74,56112.25,0.71,66.89,1.74,1.23
2,03-2018,2018,3,Total Nacional,PATATAS CONGELADAS,4270.74,5386.51,1.26,12.95,0.09,0.12
3,03-2018,2018,3,Total Nacional,PATATAS PROCESADAS,5681.63,30883.38,5.44,49.72,0.12,0.68
4,03-2018,2018,3,Total Nacional,T.HORTALIZAS FRESCAS,221895.39,401447.35,1.81,96.84,4.86,8.80
...,...,...,...,...,...,...,...,...,...,...,...
19008,11-2020,2020,11,Total Nacional,UVAS,15918.49,42755.62,2.69,0.00,0.36,0.96
19009,11-2020,2020,11,Total Nacional,KIWI,12253.15,41949.06,3.42,0.00,0.28,0.95
19010,11-2020,2020,11,Total Nacional,PATATAS FRESCAS,89679.98,76899.34,0.86,0.00,2.02,1.73
19011,11-2020,2020,11,Total Nacional,T.HORTALIZAS FRESCAS,238554.47,472592.86,1.98,0.00,5.38,10.66


### PREPROCESAMIENTO DE LOS DATOS

In [4]:
# Calcular el valor medio de todos los meses para 2018 y 2019 y rellenar con ese dato Diciembre (mes faltante)
value2018 = df_consumo.groupby(['Año','Producto','CCAA'], as_index = False)['Volumen (miles de kg)','Valor (miles de €)','Precio medio kg','Consumo per capita','Gasto per capita'].agg('mean')
value2018['Mes'] = 12
value2018['Fecha'] =  pd.to_datetime(value2018['Año'].map(str) + '-' + value2018['Mes'].map(str)).dt.strftime('%m-%Y')
value2018 = value2018[value2018['Año'] != 2020]
# Se añade al dataset de consumo
df_consumo = pd.concat([df_consumo, value2018], axis= 0)
df_consumo[df_consumo['Fecha'] == '11-2020']
# Se añade el mes de diciembre también a los productos que tienen datos hasta noviembre
value_2020 = df_consumo[df_consumo['Fecha'] == '11-2020'].copy()
value_2020['Mes'] = 12
value_2020['Fecha'] = '12-2020'
df_consumo = pd.concat([df_consumo, value_2020], axis= 0)

<ipython-input-4-d1d2d4175696>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  value2018 = df_consumo.groupby(['Año','Producto','CCAA'], as_index = False)['Volumen (miles de kg)','Valor (miles de €)','Precio medio kg','Consumo per capita','Gasto per capita'].agg('mean')


In [5]:
# Se saca una lista de todos los productos que tienen datos hasta noviembre
alimentos = value_2020['Producto'].tolist()
# Se construte un dataframe con los productos que no están en esa lista y como solo hay datos hasta junio se completa cogiendo los datos hasta diciembre del 2019
df_2020 = df_consumo[~df_consumo['Producto'].isin(alimentos)]
fechas = ['07-2019','08-2019','09-2019','10-2019','11-2019','12-2019']
df_2020 = df_2020[df_2020['Fecha'].isin(fechas)]
values2020mj = df_2020.copy()
values2020mj['Año'] = 2020
values2020mj['Fecha'] = pd.to_datetime(values2020mj['Año'].map(str) + '-' + values2020mj['Mes'].map(str)).dt.strftime('%m-%Y')
df_consumo = pd.concat([df_consumo, values2020mj], axis= 0)

In [6]:
# Se eliminan las columnas Penetración(%) y Fecha (para el modelo no interesan)
value2018['Mes'] = 12
value2018['Fecha'] =  pd.to_datetime(value2018['Año'].map(str) + '-' + value2018['Mes'].map(str)).dt.strftime('%m-%Y')
value2018 = value2018[value2018['Año'] != 2020]
df_consumo = pd.concat([df_consumo, value2018], axis= 0)
df_consumo.sort_values(by = ['Año','Mes'], inplace = True)
df_consumo.drop(columns = ['Penetración (%)','Fecha'], inplace = True)

In [7]:
# Matriz de correlación de las diferentes variables
df_consumo.corr()

,Año,Mes,Volumen (miles de kg),Valor (miles de €),Precio medio kg,Consumo per capita,Gasto per capita
Año,1.000000,-0.141820,0.005029,0.013765,0.098861,-0.035250,-0.014442
Mes,-0.141820,1.000000,-0.004896,-0.006177,0.014346,-0.033566,-0.038205
Volumen (miles de kg),0.005029,-0.004896,1.000000,0.978681,-0.092085,0.414075,0.402656
Valor (miles de €),0.013765,-0.006177,0.978681,1.000000,-0.052733,0.402096,0.412505
Precio medio kg,0.098861,0.014346,-0.092085,-0.052733,1.000000,-0.219149,-0.119766
Consumo per capita,-0.035250,-0.033566,0.414075,0.402096,-0.219149,1.000000,0.971897
Gasto per capita,-0.014442,-0.038205,0.402656,0.412505,-0.119766,0.971897,1.000000


Se observa que la variable 'Volumen (miles de kg)' (Y- target) y 'Valor (miles de €)' tienen una correlación  de ~0.98 por lo que la segunda se elimina de la X.

In [8]:
# Pasar las variables string a categóricas con el One Hot Encoder
# Se pasa la columna de CCAA
oe_style = OneHotEncoder()
oe_results = oe_style.fit_transform(df_consumo[["CCAA"]])
pd.DataFrame(oe_results.toarray(), columns=oe_style.categories_).head()
# Se pasa la columna de Producto
of_style = OneHotEncoder()
of_results = of_style.fit_transform(df_consumo[['Producto']])
pd.DataFrame(of_results.toarray(), columns=of_style.categories_)

,AGUACATE,AJOS,ALBARICOQUES,ALCACHOFAS,APIO,BERENJENAS,BROCOLI,CALABACINES,CEBOLLAS,CEREZAS,...,PUERRO,SANDIA,T.FRUTAS FRESCAS,T.HORTALIZAS FRESCAS,TOMATES,TOTAL PATATAS,UVAS,VERD./HORT. IV GAMA,VERDURAS DE HOJA,ZANAHORIAS
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
25334,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
25335,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
25336,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [9]:
# Se unen las nuevas columnas categóricas al dataframe
df_consumo_fmodel = df_consumo.join(pd.DataFrame(oe_results.toarray(), columns=oe_style.categories_))
df_consumo_fmodel = df_consumo_fmodel.join(pd.DataFrame(of_results.toarray(), columns=of_style.categories_))
df_consumo_fmodel.drop(columns=['CCAA','Producto'], inplace = True)

In [10]:
df_consumo_fmodel

,Año,Mes,Volumen (miles de kg),Valor (miles de €),Precio medio kg,Consumo per capita,Gasto per capita,"(Andalucia,)","(Aragon,)","(Asturias,)",...,"(PUERRO,)","(SANDIA,)","(T.FRUTAS FRESCAS,)","(T.HORTALIZAS FRESCAS,)","(TOMATES,)","(TOTAL PATATAS,)","(UVAS,)","(VERD./HORT. IV GAMA,)","(VERDURAS DE HOJA,)","(ZANAHORIAS,)"
0,2018,3,111978.640000,92382.140000,0.820000,2.450000,2.020000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
0,2018,12,745.715556,2380.333333,3.211111,0.085556,0.273333,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
0,2018,12,745.715556,2380.333333,3.211111,0.085556,0.273333,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2018,3,79299.740000,56112.250000,0.710000,1.740000,1.230000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018,12,139.768889,474.130000,3.404444,0.106667,0.362222,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19010,2020,12,89679.980000,76899.340000,0.860000,2.020000,1.730000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19011,2020,11,238554.470000,472592.860000,1.980000,5.380000,10.660000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19011,2020,12,238554.470000,472592.860000,1.980000,5.380000,10.660000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19012,2020,11,341351.650000,597092.660000,1.750000,7.700000,13.470000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Se separa el dataset en train y test (80%,20%)
df_consumo_fmodel.reset_index(drop=True,inplace=True)
df_consumo_fmodel = df_consumo_fmodel.sort_values(by = ['Año','Mes'])
df_consumo_fmodel.set_index('Año', inplace = True)

In [12]:
# Se selecciona la variable objetivos y los features 
Y = df_consumo_fmodel['Volumen (miles de kg)']
X = df_consumo_fmodel
X.drop(columns = ['Volumen (miles de kg)','Valor (miles de €)','Consumo per capita','Gasto per capita'], inplace = True)

In [13]:
# Se separa el dataset en train y test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

### ENTRENAMIENTO DE LOS MODELOS 

In [14]:
# Se inicializan los diferentes modelos 
xgb_model = xgb.XGBRegressor()
reg_model = lgbm.sklearn.LGBMRegressor()
cbr_model = CatBoostRegressor()

In [15]:
# Se construyen varias pipelines (es recomendable normalizar las variables antes de introducirlas en los modelos)
pipe = Pipeline([('std',StandardScaler()),('model',xgb_model)])
pipe2 = Pipeline([('std',StandardScaler()),('model',reg_model)])
pipe3 = Pipeline([('std',StandardScaler()),('model',cbr_model)])

In [16]:
# Se entrenan los 3 modelos 
pipe.fit(X_train, y_train)
pipe2.fit(X_train, y_train)
pipe3.fit(X_train, y_train)

Learning rate set to 0.066684
0:	learn: 17742.2281083	total: 146ms	remaining: 2m 25s
1:	learn: 17634.5067802	total: 152ms	remaining: 1m 15s
2:	learn: 17535.6621146	total: 158ms	remaining: 52.4s
3:	learn: 17428.7135657	total: 163ms	remaining: 40.7s
4:	learn: 17332.4024829	total: 169ms	remaining: 33.6s
5:	learn: 17251.1733578	total: 175ms	remaining: 29s
6:	learn: 17171.9136087	total: 181ms	remaining: 25.7s
7:	learn: 17101.0901791	total: 186ms	remaining: 23.1s
8:	learn: 17043.4144108	total: 192ms	remaining: 21.1s
9:	learn: 16974.1244494	total: 197ms	remaining: 19.5s
10:	learn: 16908.8340396	total: 202ms	remaining: 18.2s
11:	learn: 16855.2835437	total: 208ms	remaining: 17.1s
12:	learn: 16808.6356328	total: 213ms	remaining: 16.2s
13:	learn: 16767.9784523	total: 219ms	remaining: 15.4s
14:	learn: 16723.1006887	total: 224ms	remaining: 14.7s
15:	learn: 16678.4860981	total: 229ms	remaining: 14.1s
16:	learn: 16643.8944781	total: 234ms	remaining: 13.5s
17:	learn: 16626.3285569	total: 239ms	remaini

164:	learn: 14505.4185356	total: 939ms	remaining: 4.75s
165:	learn: 14495.5700893	total: 943ms	remaining: 4.74s
166:	learn: 14491.0543948	total: 948ms	remaining: 4.73s
167:	learn: 14477.6219851	total: 954ms	remaining: 4.72s
168:	learn: 14460.8677419	total: 959ms	remaining: 4.72s
169:	learn: 14454.1667707	total: 964ms	remaining: 4.71s
170:	learn: 14448.3626058	total: 969ms	remaining: 4.7s
171:	learn: 14436.3828359	total: 975ms	remaining: 4.69s
172:	learn: 14420.7175642	total: 980ms	remaining: 4.68s
173:	learn: 14411.3187916	total: 984ms	remaining: 4.67s
174:	learn: 14407.1802295	total: 988ms	remaining: 4.66s
175:	learn: 14393.8454011	total: 993ms	remaining: 4.65s
176:	learn: 14385.8833522	total: 997ms	remaining: 4.64s
177:	learn: 14379.8688945	total: 1s	remaining: 4.62s
178:	learn: 14367.5816482	total: 1s	remaining: 4.61s
179:	learn: 14352.4478591	total: 1.01s	remaining: 4.6s
180:	learn: 14342.3305028	total: 1.01s	remaining: 4.59s
181:	learn: 14332.2689059	total: 1.02s	remaining: 4.58s


331:	learn: 13178.1386171	total: 1.8s	remaining: 3.62s
332:	learn: 13170.8014154	total: 1.81s	remaining: 3.62s
333:	learn: 13160.1419526	total: 1.81s	remaining: 3.61s
334:	learn: 13155.5677784	total: 1.82s	remaining: 3.61s
335:	learn: 13152.3919661	total: 1.82s	remaining: 3.6s
336:	learn: 13146.3722716	total: 1.83s	remaining: 3.6s
337:	learn: 13141.6565027	total: 1.83s	remaining: 3.59s
338:	learn: 13131.0787433	total: 1.84s	remaining: 3.58s
339:	learn: 13123.7349409	total: 1.84s	remaining: 3.58s
340:	learn: 13117.3016167	total: 1.85s	remaining: 3.57s
341:	learn: 13106.9820633	total: 1.85s	remaining: 3.56s
342:	learn: 13098.5569202	total: 1.86s	remaining: 3.56s
343:	learn: 13091.5258493	total: 1.86s	remaining: 3.55s
344:	learn: 13086.8704189	total: 1.87s	remaining: 3.54s
345:	learn: 13080.9493511	total: 1.87s	remaining: 3.54s
346:	learn: 13073.2656832	total: 1.88s	remaining: 3.53s
347:	learn: 13059.8243165	total: 1.88s	remaining: 3.52s
348:	learn: 13052.9613399	total: 1.89s	remaining: 3

508:	learn: 12211.3041967	total: 2.6s	remaining: 2.51s
509:	learn: 12204.2833980	total: 2.6s	remaining: 2.5s
510:	learn: 12202.3972699	total: 2.61s	remaining: 2.5s
511:	learn: 12191.4968478	total: 2.61s	remaining: 2.49s
512:	learn: 12185.8452375	total: 2.62s	remaining: 2.48s
513:	learn: 12184.2538223	total: 2.62s	remaining: 2.48s
514:	learn: 12171.7863907	total: 2.63s	remaining: 2.48s
515:	learn: 12161.6920152	total: 2.63s	remaining: 2.47s
516:	learn: 12156.1784339	total: 2.64s	remaining: 2.46s
517:	learn: 12153.2382960	total: 2.64s	remaining: 2.46s
518:	learn: 12146.4231289	total: 2.65s	remaining: 2.45s
519:	learn: 12136.0370277	total: 2.65s	remaining: 2.45s
520:	learn: 12132.1022758	total: 2.66s	remaining: 2.44s
521:	learn: 12127.7390998	total: 2.66s	remaining: 2.44s
522:	learn: 12123.9127779	total: 2.66s	remaining: 2.43s
523:	learn: 12121.3491051	total: 2.67s	remaining: 2.42s
524:	learn: 12113.6463030	total: 2.67s	remaining: 2.42s
525:	learn: 12107.0442649	total: 2.68s	remaining: 2.

680:	learn: 11336.7613170	total: 3.39s	remaining: 1.59s
681:	learn: 11329.4427866	total: 3.4s	remaining: 1.58s
682:	learn: 11322.8444965	total: 3.4s	remaining: 1.58s
683:	learn: 11318.0421077	total: 3.41s	remaining: 1.57s
684:	learn: 11316.6725292	total: 3.41s	remaining: 1.57s
685:	learn: 11310.1871666	total: 3.42s	remaining: 1.56s
686:	learn: 11305.3716743	total: 3.42s	remaining: 1.56s
687:	learn: 11302.0803915	total: 3.43s	remaining: 1.55s
688:	learn: 11298.5943973	total: 3.43s	remaining: 1.55s
689:	learn: 11297.1632099	total: 3.44s	remaining: 1.54s
690:	learn: 11280.0816483	total: 3.44s	remaining: 1.54s
691:	learn: 11277.6869345	total: 3.44s	remaining: 1.53s
692:	learn: 11271.1542366	total: 3.45s	remaining: 1.53s
693:	learn: 11266.5938896	total: 3.45s	remaining: 1.52s
694:	learn: 11263.8122454	total: 3.46s	remaining: 1.52s
695:	learn: 11261.3414069	total: 3.46s	remaining: 1.51s
696:	learn: 11258.6541978	total: 3.47s	remaining: 1.51s
697:	learn: 11256.8193833	total: 3.47s	remaining: 

844:	learn: 10656.2347014	total: 4.15s	remaining: 761ms
845:	learn: 10643.1989860	total: 4.15s	remaining: 756ms
846:	learn: 10642.5500117	total: 4.16s	remaining: 751ms
847:	learn: 10632.4331684	total: 4.16s	remaining: 746ms
848:	learn: 10628.7364537	total: 4.17s	remaining: 741ms
849:	learn: 10622.2993501	total: 4.17s	remaining: 736ms
850:	learn: 10618.7423542	total: 4.18s	remaining: 731ms
851:	learn: 10618.1176294	total: 4.18s	remaining: 726ms
852:	learn: 10612.3359495	total: 4.19s	remaining: 721ms
853:	learn: 10611.7357909	total: 4.19s	remaining: 716ms
854:	learn: 10608.3118735	total: 4.2s	remaining: 712ms
855:	learn: 10603.8030062	total: 4.2s	remaining: 707ms
856:	learn: 10597.7791359	total: 4.2s	remaining: 702ms
857:	learn: 10594.5804480	total: 4.21s	remaining: 697ms
858:	learn: 10591.5026263	total: 4.21s	remaining: 692ms
859:	learn: 10582.3146089	total: 4.22s	remaining: 687ms
860:	learn: 10580.6048543	total: 4.22s	remaining: 682ms
861:	learn: 10576.9864244	total: 4.23s	remaining: 6

Pipeline(steps=[('std', StandardScaler()),
                ('model',
                 <catboost.core.CatBoostRegressor object at 0x000001AE75DA4130>)])

In [17]:
# Se calcula el score de los 3 modelos (en los modelos de regresión se obtiene el R2 score)
print('El score del XGBoost es: ' , "{:.3f}".format(pipe.score(X_test,y_test)))
print('El score del LGBMRegressor es: ', "{:.3f}".format(pipe2.score(X_test,y_test)))
print('El score del CatBoost es: ' , "{:.3f}".format(pipe3.score(X_test,y_test)))

El score del XGBoost es:  0.124
El score del LGBMRegressor es:  0.123
El score del CatBoost es:  0.213



*El modelo que más score devuelve es el CatBoost, lo cual es lógico ya que funciona muy bien cuando se tienen varias variables categóricas como ocurre en este caso (producto y CCAA). El score de r2 es bastante bajo pero superior a 0. Teniendo en cuenta que se ha entrenado con valores de 3 años y que el año 2020 es prácticamente un outlier (valores fuera de lo normal), el resultado es aceptable. Igualmente, se puede mejorar realizando la búsqueda de metaparámetros*

In [18]:
# Se calcula el score por validación cruzada
scores = cross_val_score(pipe3, X,Y, cv=5, scoring='r2')
score = np.mean(scores)

Learning rate set to 0.066684
0:	learn: 17793.3125445	total: 5.62ms	remaining: 5.61s
1:	learn: 17751.7266828	total: 10.5ms	remaining: 5.23s
2:	learn: 17706.6431855	total: 15.4ms	remaining: 5.11s
3:	learn: 17667.6958673	total: 20.6ms	remaining: 5.12s
4:	learn: 17626.4528540	total: 25.6ms	remaining: 5.09s
5:	learn: 17591.2296864	total: 30.6ms	remaining: 5.07s
6:	learn: 17555.2169827	total: 35.8ms	remaining: 5.07s
7:	learn: 17524.0453729	total: 40.7ms	remaining: 5.04s
8:	learn: 17495.0997775	total: 45.7ms	remaining: 5.03s
9:	learn: 17469.8401583	total: 50.8ms	remaining: 5.03s
10:	learn: 17438.4887029	total: 56.3ms	remaining: 5.06s
11:	learn: 17413.0876761	total: 61.4ms	remaining: 5.05s
12:	learn: 17384.1508102	total: 66.6ms	remaining: 5.06s
13:	learn: 17355.6654193	total: 71.9ms	remaining: 5.06s
14:	learn: 17319.7288564	total: 77.2ms	remaining: 5.07s
15:	learn: 17295.9380045	total: 82.5ms	remaining: 5.08s
16:	learn: 17275.8874642	total: 87.7ms	remaining: 5.07s
17:	learn: 17262.6552963	tot

155:	learn: 15315.8952254	total: 766ms	remaining: 4.14s
156:	learn: 15307.1806877	total: 771ms	remaining: 4.14s
157:	learn: 15295.2537595	total: 777ms	remaining: 4.14s
158:	learn: 15276.9797142	total: 782ms	remaining: 4.14s
159:	learn: 15261.8152131	total: 787ms	remaining: 4.13s
160:	learn: 15247.9616107	total: 792ms	remaining: 4.13s
161:	learn: 15237.3208808	total: 798ms	remaining: 4.13s
162:	learn: 15220.1998693	total: 803ms	remaining: 4.12s
163:	learn: 15205.7395191	total: 808ms	remaining: 4.12s
164:	learn: 15192.4746172	total: 814ms	remaining: 4.12s
165:	learn: 15179.6241314	total: 818ms	remaining: 4.11s
166:	learn: 15175.1749547	total: 823ms	remaining: 4.11s
167:	learn: 15158.6176928	total: 828ms	remaining: 4.1s
168:	learn: 15151.1253185	total: 833ms	remaining: 4.09s
169:	learn: 15137.1734418	total: 838ms	remaining: 4.09s
170:	learn: 15129.1215416	total: 842ms	remaining: 4.08s
171:	learn: 15112.9345162	total: 848ms	remaining: 4.08s
172:	learn: 15096.3503433	total: 852ms	remaining:

319:	learn: 13796.9987956	total: 1.56s	remaining: 3.31s
320:	learn: 13788.4331199	total: 1.56s	remaining: 3.3s
321:	learn: 13783.9431181	total: 1.57s	remaining: 3.3s
322:	learn: 13772.1579130	total: 1.57s	remaining: 3.3s
323:	learn: 13764.3746663	total: 1.58s	remaining: 3.29s
324:	learn: 13753.8406507	total: 1.58s	remaining: 3.29s
325:	learn: 13745.7462868	total: 1.59s	remaining: 3.28s
326:	learn: 13742.0275240	total: 1.59s	remaining: 3.28s
327:	learn: 13733.7792618	total: 1.6s	remaining: 3.27s
328:	learn: 13728.4789991	total: 1.6s	remaining: 3.27s
329:	learn: 13723.5601083	total: 1.61s	remaining: 3.26s
330:	learn: 13708.7786271	total: 1.61s	remaining: 3.26s
331:	learn: 13702.1786400	total: 1.62s	remaining: 3.25s
332:	learn: 13692.5661182	total: 1.62s	remaining: 3.25s
333:	learn: 13686.2130361	total: 1.63s	remaining: 3.24s
334:	learn: 13674.3152565	total: 1.63s	remaining: 3.23s
335:	learn: 13671.5006874	total: 1.63s	remaining: 3.23s
336:	learn: 13665.1521127	total: 1.64s	remaining: 3.2

467:	learn: 13006.7354353	total: 2.27s	remaining: 2.58s
468:	learn: 12998.7354857	total: 2.28s	remaining: 2.58s
469:	learn: 12997.3730489	total: 2.28s	remaining: 2.58s
470:	learn: 12991.4108826	total: 2.29s	remaining: 2.57s
471:	learn: 12990.8292273	total: 2.29s	remaining: 2.57s
472:	learn: 12977.5714690	total: 2.3s	remaining: 2.56s
473:	learn: 12971.0482546	total: 2.31s	remaining: 2.56s
474:	learn: 12969.4752528	total: 2.31s	remaining: 2.56s
475:	learn: 12964.6094832	total: 2.32s	remaining: 2.55s
476:	learn: 12956.6387141	total: 2.32s	remaining: 2.55s
477:	learn: 12955.1867809	total: 2.33s	remaining: 2.54s
478:	learn: 12948.0433826	total: 2.33s	remaining: 2.54s
479:	learn: 12944.1313383	total: 2.34s	remaining: 2.53s
480:	learn: 12936.7821615	total: 2.34s	remaining: 2.53s
481:	learn: 12932.7441632	total: 2.35s	remaining: 2.52s
482:	learn: 12928.6899698	total: 2.35s	remaining: 2.52s
483:	learn: 12921.8497856	total: 2.36s	remaining: 2.51s
484:	learn: 12915.9225937	total: 2.36s	remaining:

616:	learn: 12344.4061965	total: 2.98s	remaining: 1.85s
617:	learn: 12343.6031274	total: 2.98s	remaining: 1.84s
618:	learn: 12341.2711247	total: 2.99s	remaining: 1.84s
619:	learn: 12338.4383029	total: 2.99s	remaining: 1.83s
620:	learn: 12331.3302307	total: 3s	remaining: 1.83s
621:	learn: 12318.8944723	total: 3s	remaining: 1.82s
622:	learn: 12316.0421096	total: 3.01s	remaining: 1.82s
623:	learn: 12315.6402383	total: 3.01s	remaining: 1.81s
624:	learn: 12305.9844307	total: 3.02s	remaining: 1.81s
625:	learn: 12293.8401348	total: 3.02s	remaining: 1.8s
626:	learn: 12287.9731603	total: 3.02s	remaining: 1.8s
627:	learn: 12286.5314156	total: 3.03s	remaining: 1.79s
628:	learn: 12284.5996286	total: 3.03s	remaining: 1.79s
629:	learn: 12280.9879164	total: 3.04s	remaining: 1.78s
630:	learn: 12279.1814285	total: 3.04s	remaining: 1.78s
631:	learn: 12274.5350663	total: 3.04s	remaining: 1.77s
632:	learn: 12270.0749090	total: 3.05s	remaining: 1.77s
633:	learn: 12267.4457167	total: 3.05s	remaining: 1.76s


...

171:	learn: 12439.8063393	total: 835ms	remaining: 4.02s
172:	learn: 12428.0170366	total: 841ms	remaining: 4.02s
173:	learn: 12417.3459274	total: 847ms	remaining: 4.02s
174:	learn: 12409.8617815	total: 852ms	remaining: 4.02s
175:	learn: 12398.2119844	total: 857ms	remaining: 4.01s
176:	learn: 12382.3880099	total: 863ms	remaining: 4.01s
177:	learn: 12373.4629879	total: 867ms	remaining: 4.01s
178:	learn: 12365.1746849	total: 872ms	remaining: 4s
179:	learn: 12345.6160704	total: 877ms	remaining: 3.99s
180:	learn: 12328.9327809	total: 882ms	remaining: 3.99s
181:	learn: 12324.9044481	total: 888ms	remaining: 3.99s
182:	learn: 12317.5605983	total: 893ms	remaining: 3.99s
183:	learn: 12305.3233081	total: 898ms	remaining: 3.98s
184:	learn: 12300.3168005	total: 903ms	remaining: 3.98s
185:	learn: 12296.4411040	total: 908ms	remaining: 3.97s
186:	learn: 12292.6353412	total: 912ms	remaining: 3.97s
187:	learn: 12276.8380891	total: 916ms	remaining: 3.96s
188:	learn: 12264.3596918	total: 920ms	remaining: 3

354:	learn: 11072.6613138	total: 1.68s	remaining: 3.05s
355:	learn: 11066.8843751	total: 1.69s	remaining: 3.05s
356:	learn: 11059.0571311	total: 1.69s	remaining: 3.04s
357:	learn: 11054.3966123	total: 1.7s	remaining: 3.04s
358:	learn: 11048.8429487	total: 1.7s	remaining: 3.04s
359:	learn: 11044.7362051	total: 1.71s	remaining: 3.04s
360:	learn: 11034.9138613	total: 1.71s	remaining: 3.03s
361:	learn: 11027.2334313	total: 1.72s	remaining: 3.03s
362:	learn: 11021.5858747	total: 1.72s	remaining: 3.02s
363:	learn: 11012.4896339	total: 1.73s	remaining: 3.02s
364:	learn: 11010.2924242	total: 1.73s	remaining: 3.01s
365:	learn: 11006.0208051	total: 1.74s	remaining: 3.01s
366:	learn: 11001.4739846	total: 1.74s	remaining: 3s
367:	learn: 10998.0805516	total: 1.74s	remaining: 3s
368:	learn: 10991.6650467	total: 1.75s	remaining: 2.99s
369:	learn: 10989.2123187	total: 1.75s	remaining: 2.99s
370:	learn: 10983.9330642	total: 1.76s	remaining: 2.98s
371:	learn: 10977.7610459	total: 1.76s	remaining: 2.98s


520:	learn: 10203.1302347	total: 2.47s	remaining: 2.27s
521:	learn: 10198.5248390	total: 2.48s	remaining: 2.27s
522:	learn: 10195.3398033	total: 2.48s	remaining: 2.27s
523:	learn: 10191.7831484	total: 2.49s	remaining: 2.26s
524:	learn: 10188.3731695	total: 2.49s	remaining: 2.26s
525:	learn: 10182.3976832	total: 2.5s	remaining: 2.25s
526:	learn: 10174.2560901	total: 2.5s	remaining: 2.25s
527:	learn: 10168.1281282	total: 2.51s	remaining: 2.24s
528:	learn: 10162.8874200	total: 2.51s	remaining: 2.24s
529:	learn: 10161.0816261	total: 2.52s	remaining: 2.23s
530:	learn: 10157.8040449	total: 2.52s	remaining: 2.23s
531:	learn: 10149.0708282	total: 2.55s	remaining: 2.24s
532:	learn: 10143.3691698	total: 2.56s	remaining: 2.24s
533:	learn: 10140.8916198	total: 2.56s	remaining: 2.23s
534:	learn: 10134.4293724	total: 2.57s	remaining: 2.23s
535:	learn: 10126.1589427	total: 2.57s	remaining: 2.23s
536:	learn: 10119.6675745	total: 2.58s	remaining: 2.22s
537:	learn: 10111.1440331	total: 2.58s	remaining: 

688:	learn: 9442.1425875	total: 3.29s	remaining: 1.48s
689:	learn: 9436.5387160	total: 3.29s	remaining: 1.48s
690:	learn: 9430.8107475	total: 3.3s	remaining: 1.47s
691:	learn: 9425.0603924	total: 3.3s	remaining: 1.47s
692:	learn: 9424.0456783	total: 3.31s	remaining: 1.46s
693:	learn: 9421.7359504	total: 3.31s	remaining: 1.46s
694:	learn: 9417.9519953	total: 3.31s	remaining: 1.46s
695:	learn: 9414.1332041	total: 3.32s	remaining: 1.45s
696:	learn: 9411.8270703	total: 3.33s	remaining: 1.45s
697:	learn: 9407.8529072	total: 3.33s	remaining: 1.44s
698:	learn: 9400.4440470	total: 3.34s	remaining: 1.44s
699:	learn: 9396.7001677	total: 3.34s	remaining: 1.43s
700:	learn: 9390.7452328	total: 3.35s	remaining: 1.43s
701:	learn: 9386.7107588	total: 3.35s	remaining: 1.42s
702:	learn: 9380.7930898	total: 3.35s	remaining: 1.42s
703:	learn: 9370.5615996	total: 3.36s	remaining: 1.41s
704:	learn: 9366.8804890	total: 3.36s	remaining: 1.41s
705:	learn: 9361.8259039	total: 3.37s	remaining: 1.4s
706:	learn: 9

859:	learn: 8629.3714642	total: 4.09s	remaining: 666ms
860:	learn: 8625.6062491	total: 4.1s	remaining: 662ms
861:	learn: 8621.0255060	total: 4.1s	remaining: 657ms
862:	learn: 8615.2147458	total: 4.11s	remaining: 652ms
863:	learn: 8611.1816501	total: 4.11s	remaining: 647ms
864:	learn: 8607.2862268	total: 4.12s	remaining: 643ms
865:	learn: 8602.9033947	total: 4.12s	remaining: 638ms
866:	learn: 8601.4831696	total: 4.13s	remaining: 633ms
867:	learn: 8597.9569656	total: 4.13s	remaining: 628ms
868:	learn: 8594.7976597	total: 4.13s	remaining: 623ms
869:	learn: 8593.3451355	total: 4.14s	remaining: 619ms
870:	learn: 8592.1809345	total: 4.15s	remaining: 614ms
871:	learn: 8588.6767385	total: 4.15s	remaining: 609ms
872:	learn: 8587.5899737	total: 4.16s	remaining: 605ms
873:	learn: 8583.4310404	total: 4.16s	remaining: 600ms
874:	learn: 8580.3787522	total: 4.17s	remaining: 595ms
875:	learn: 8576.7259383	total: 4.17s	remaining: 590ms
876:	learn: 8573.1961938	total: 4.17s	remaining: 586ms
877:	learn: 

In [19]:
print("El score del CatBoost con validación cruzada es: ", round(score,3))


El score del CatBoost con validación cruzada es:  -0.171


In [20]:
# Se realiza la búsqueda de metaparámetros
param_grid = {
    'model__max_depth': [2, 3, 5, 7, 10],
    'model__n_estimators': [10, 100, 500],
}

# Se define el grid para el modelo de LGBM Regressor (pipe2) , 5 iteraciones y el score del modelo (r2)
grid = GridSearchCV(pipe3, param_grid, cv=5, n_jobs=-1, scoring='r2')

In [21]:
# Se ajusta el modelo
grid.fit(X, Y)

Learning rate set to 0.117505
0:	learn: 17686.4211102	total: 3.26ms	remaining: 1.62s
1:	learn: 17597.2247124	total: 6.03ms	remaining: 1.5s
2:	learn: 17553.0412565	total: 8.71ms	remaining: 1.44s
3:	learn: 17485.5214262	total: 11.3ms	remaining: 1.41s
4:	learn: 17456.8063801	total: 14.2ms	remaining: 1.41s
5:	learn: 17404.7731601	total: 17ms	remaining: 1.4s
6:	learn: 17385.5602122	total: 19.6ms	remaining: 1.38s
7:	learn: 17362.1735742	total: 23.4ms	remaining: 1.44s
8:	learn: 17346.0194830	total: 26.4ms	remaining: 1.44s
9:	learn: 17333.8707075	total: 29.3ms	remaining: 1.44s
10:	learn: 17294.0262326	total: 33ms	remaining: 1.47s
11:	learn: 17283.1741997	total: 36.2ms	remaining: 1.47s
12:	learn: 17254.0532037	total: 39.2ms	remaining: 1.47s
13:	learn: 17239.9733206	total: 42ms	remaining: 1.46s
14:	learn: 17230.2083248	total: 45.4ms	remaining: 1.47s
15:	learn: 17218.8107370	total: 48.6ms	remaining: 1.47s
16:	learn: 17210.9699353	total: 51.7ms	remaining: 1.47s
17:	learn: 17201.9947092	total: 55ms

164:	learn: 16679.4918268	total: 513ms	remaining: 1.04s
165:	learn: 16677.5170448	total: 516ms	remaining: 1.04s
166:	learn: 16677.0718817	total: 520ms	remaining: 1.04s
167:	learn: 16674.9486955	total: 524ms	remaining: 1.03s
168:	learn: 16674.2609850	total: 527ms	remaining: 1.03s
169:	learn: 16673.8981141	total: 530ms	remaining: 1.03s
170:	learn: 16672.5036568	total: 535ms	remaining: 1.03s
171:	learn: 16671.5518131	total: 539ms	remaining: 1.03s
172:	learn: 16671.0347941	total: 543ms	remaining: 1.02s
173:	learn: 16670.2133363	total: 546ms	remaining: 1.02s
174:	learn: 16669.5426144	total: 550ms	remaining: 1.02s
175:	learn: 16669.1252676	total: 554ms	remaining: 1.02s
176:	learn: 16668.5497560	total: 557ms	remaining: 1.02s
177:	learn: 16667.3957298	total: 561ms	remaining: 1.01s
178:	learn: 16666.6302370	total: 566ms	remaining: 1.01s
179:	learn: 16665.9392957	total: 570ms	remaining: 1.01s
180:	learn: 16665.4658017	total: 573ms	remaining: 1.01s
181:	learn: 16664.5055680	total: 579ms	remaining

350:	learn: 16526.3532379	total: 1.06s	remaining: 450ms
351:	learn: 16525.5007696	total: 1.06s	remaining: 447ms
352:	learn: 16524.8291333	total: 1.06s	remaining: 444ms
353:	learn: 16524.2815497	total: 1.07s	remaining: 441ms
354:	learn: 16523.0346666	total: 1.07s	remaining: 438ms
355:	learn: 16522.1915531	total: 1.07s	remaining: 434ms
356:	learn: 16521.8148042	total: 1.08s	remaining: 431ms
357:	learn: 16520.4928567	total: 1.08s	remaining: 428ms
358:	learn: 16520.3185298	total: 1.08s	remaining: 425ms
359:	learn: 16520.1071607	total: 1.08s	remaining: 422ms
360:	learn: 16519.2502454	total: 1.09s	remaining: 418ms
361:	learn: 16518.8511744	total: 1.09s	remaining: 415ms
362:	learn: 16518.3425728	total: 1.09s	remaining: 412ms
363:	learn: 16518.1282329	total: 1.09s	remaining: 409ms
364:	learn: 16516.8733133	total: 1.09s	remaining: 405ms
365:	learn: 16515.5933134	total: 1.1s	remaining: 402ms
366:	learn: 16515.4495132	total: 1.1s	remaining: 399ms
367:	learn: 16514.1286149	total: 1.1s	remaining: 3

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('std', StandardScaler()),
                                       ('model',
                                        <catboost.core.CatBoostRegressor object at 0x000001AE75DA4130>)]),
             n_jobs=-1,
             param_grid={'model__max_depth': [2, 3, 5, 7, 10],
                         'model__n_estimators': [10, 100, 500]},
             scoring='r2')

In [22]:
# Se calcula la media de los score y la desviación típica
mean_score = grid.cv_results_["mean_test_score"][grid.best_index_]
std_score = grid.cv_results_["std_test_score"][grid.best_index_]

print("Los metaparámetros óptimos para el modelo son :" , grid.best_params_)
print("El r2 score medio de validación cruzada es:" , round(mean_score,3))
print("La desviación media de validación cruzada es:", round(std_score,3))

Los metaparámetros óptimos para el modelo son : {'model__max_depth': 2, 'model__n_estimators': 500}
El r2 score medio de validación cruzada es: 0.011
La desviación media de validación cruzada es: 0.081


*El valor de r2 con el modelo optimizado es muy bajo. Esto puede deberse a que se ha entrenado con todos los datos sin diferenciar el lugar de compra del cual proceden los productos. Por eso es fundamental realizar más modelos para los datos pero segmentados por lugar de compra ya que cada uno tiene una tendencia diferente. Para ello en el análisis 5, se realiza un modelo para los datos de consumo por internet.*

Para visualizar el resultado, acudir a: https://datacrop.es/reto-analisis-5/